# Practical Application III: Comparing Classifiers

**Overview**: In this practical application, your goal is to compare the performance of the classifiers we encountered in this section, namely K Nearest Neighbor, Logistic Regression, Decision Trees, and Support Vector Machines.  We will utilize a dataset related to marketing bank products over the telephone.  



### Getting Started

Our dataset comes from the UCI Machine Learning repository [link](https://archive.ics.uci.edu/ml/datasets/bank+marketing).  The data is from a Portugese banking institution and is a collection of the results of multiple marketing campaigns.  We will make use of the article accompanying the dataset [here](CRISP-DM-BANK.pdf) for more information on the data and features.



### Problem 1: Understanding the Data

To gain a better understanding of the data, please read the information provided in the UCI link above, and examine the **Materials and Methods** section of the paper.  How many marketing campaigns does this data represent?

### The dataset collected is related to 17 campaigns that occured at different times over the course of multiple years.


### Problem 2: Read in the Data

Use pandas to read in the dataset `bank-additional-full.csv` and assign to a meaningful variable name.

In [158]:
import pandas as pd
%config Completer.use_jedi = False
from sklearn.svm import SVC
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC

In [4]:
df = pd.read_csv('data/bank-additional-full.csv', sep = ';')

In [5]:
df.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


### Problem 3: Understanding the Features


Examine the data description below, and determine if any of the features are missing values or need to be coerced to a different data type.


```
Input variables:
# bank client data:
1 - age (numeric)
2 - job : type of job (categorical: 'admin.','blue-collar','entrepreneur','housemaid','management','retired','self-employed','services','student','technician','unemployed','unknown')
3 - marital : marital status (categorical: 'divorced','married','single','unknown'; note: 'divorced' means divorced or widowed)
4 - education (categorical: 'basic.4y','basic.6y','basic.9y','high.school','illiterate','professional.course','university.degree','unknown')
5 - default: has credit in default? (categorical: 'no','yes','unknown')
6 - housing: has housing loan? (categorical: 'no','yes','unknown')
7 - loan: has personal loan? (categorical: 'no','yes','unknown')
# related with the last contact of the current campaign:
8 - contact: contact communication type (categorical: 'cellular','telephone')
9 - month: last contact month of year (categorical: 'jan', 'feb', 'mar', ..., 'nov', 'dec')
10 - day_of_week: last contact day of the week (categorical: 'mon','tue','wed','thu','fri')
11 - duration: last contact duration, in seconds (numeric). Important note: this attribute highly affects the output target (e.g., if duration=0 then y='no'). Yet, the duration is not known before a call is performed. Also, after the end of the call y is obviously known. Thus, this input should only be included for benchmark purposes and should be discarded if the intention is to have a realistic predictive model.
# other attributes:
12 - campaign: number of contacts performed during this campaign and for this client (numeric, includes last contact)
13 - pdays: number of days that passed by after the client was last contacted from a previous campaign (numeric; 999 means client was not previously contacted)
14 - previous: number of contacts performed before this campaign and for this client (numeric)
15 - poutcome: outcome of the previous marketing campaign (categorical: 'failure','nonexistent','success')
# social and economic context attributes
16 - emp.var.rate: employment variation rate - quarterly indicator (numeric)
17 - cons.price.idx: consumer price index - monthly indicator (numeric)
18 - cons.conf.idx: consumer confidence index - monthly indicator (numeric)
19 - euribor3m: euribor 3 month rate - daily indicator (numeric)
20 - nr.employed: number of employees - quarterly indicator (numeric)

Output variable (desired target):
21 - y - has the client subscribed a term deposit? (binary: 'yes','no')
```



In [186]:
df['previous'].value_counts()

0    35563
1     4561
2      754
3      216
4       70
5       18
6        5
7        1
Name: previous, dtype: int64

### Problem 4: Understanding the Task

After examining the description and data, your goal now is to clearly state the *Business Objective* of the task.  State the objective below.

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             41188 non-null  int64  
 1   job             41188 non-null  object 
 2   marital         41188 non-null  object 
 3   education       41188 non-null  object 
 4   default         41188 non-null  object 
 5   housing         41188 non-null  object 
 6   loan            41188 non-null  object 
 7   contact         41188 non-null  object 
 8   month           41188 non-null  object 
 9   day_of_week     41188 non-null  object 
 10  duration        41188 non-null  int64  
 11  campaign        41188 non-null  int64  
 12  pdays           41188 non-null  int64  
 13  previous        41188 non-null  int64  
 14  poutcome        41188 non-null  object 
 15  emp.var.rate    41188 non-null  float64
 16  cons.price.idx  41188 non-null  float64
 17  cons.conf.idx   41188 non-null 

### We would like to demonstrate a clear relationship between the seven selected variables and the wether or not a customer is likely to accept a marketing campaign. This would allow for a more efficient selection of customers to recieve marketing calls saving money and time on both ends.

### Problem 5: Engineering Features

Now that you understand your business objective, we will build a basic model to get started.  Before we can do this, we must work to encode the data.  Using just the bank information features (columns 1 - 7), prepare the features and target column for modeling with appropriate encoding and transformations.

In [6]:
df.isnull().sum()


age               0
job               0
marital           0
education         0
default           0
housing           0
loan              0
contact           0
month             0
day_of_week       0
duration          0
campaign          0
pdays             0
previous          0
poutcome          0
emp.var.rate      0
cons.price.idx    0
cons.conf.idx     0
euribor3m         0
nr.employed       0
y                 0
dtype: int64

In [31]:
data=df.iloc[:,[-1,0,1,2,3,4,5,6]]
data['age']=(data.age-data.age.mean())/data.age.std()
data['y']=data.y.map({'no':0,'yes':1})


0        1.533016
1        1.628973
2       -0.290182
3       -0.002309
4        1.533016
           ...   
41183    3.164298
41184    0.573438
41185    1.533016
41186    0.381522
41187    3.260256
Name: age, Length: 41188, dtype: float64

In [142]:
#the following code is simply manually done one hot encoding.

scaled_data=pd.get_dummies(data=data, columns=['job', 'marital', 'education', 'default', 'housing',
       'loan'])
scaled_data.query('y==1')


,y,age,job_admin.,job_blue-collar,job_entrepreneur,job_housemaid,job_management,job_retired,job_self-employed,job_services,...,education_unknown,default_no,default_unknown,default_yes,housing_no,housing_unknown,housing_yes,loan_no,loan_unknown,loan_yes
75,1,0.093649,0,1,0,0,0,0,0,0,...,0,0,1,0,0,0,1,1,0,0
83,1,0.861311,0,0,1,0,0,0,0,0,...,0,0,1,0,0,0,1,1,0,0
88,1,0.861311,0,0,0,0,0,0,0,0,...,0,1,0,0,1,0,0,1,0,0
129,1,0.093649,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,1,1,0,0
139,1,0.477480,0,1,0,0,0,0,0,0,...,0,0,1,0,0,0,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41174,1,2.108762,0,0,0,0,0,1,0,0,...,0,1,0,0,0,0,1,1,0,0
41178,1,2.108762,0,0,0,0,0,1,0,0,...,0,1,0,0,1,0,0,1,0,0
41181,1,-0.290182,1,0,0,0,0,0,0,0,...,0,1,0,0,0,0,1,1,0,0
41183,1,3.164298,0,0,0,0,0,1,0,0,...,0,1,0,0,0,0,1,1,0,0


In [143]:

scaled_data.query('y==1')
scaled_data.query('y==0').sample(n=4640)

,y,age,job_admin.,job_blue-collar,job_entrepreneur,job_housemaid,job_management,job_retired,job_self-employed,job_services,...,education_unknown,default_no,default_unknown,default_yes,housing_no,housing_unknown,housing_yes,loan_no,loan_unknown,loan_yes
7800,0,1.437058,0,0,0,0,0,0,0,0,...,0,0,1,0,1,0,0,1,0,0
2877,0,-0.482098,0,0,0,0,0,0,0,0,...,0,1,0,0,1,0,0,1,0,0
6491,0,-0.482098,0,0,0,0,1,0,0,0,...,0,1,0,0,0,0,1,0,0,1
1795,0,1.149185,1,0,0,0,0,0,0,0,...,0,1,0,0,1,0,0,1,0,0
37560,0,-1.441675,0,0,0,0,0,0,0,0,...,0,1,0,0,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29894,0,-0.002309,0,0,0,0,1,0,0,0,...,0,1,0,0,1,0,0,1,0,0
29708,0,-0.002309,0,0,0,0,0,0,1,0,...,0,1,0,0,0,0,1,1,0,0
33399,0,-0.769971,0,1,0,0,0,0,0,0,...,0,1,0,0,0,0,1,1,0,0
28125,0,0.477480,1,0,0,0,0,0,0,0,...,0,1,0,0,0,0,1,1,0,0


In [144]:
#the following code creates a perfectly balanced dataset with an equal number of customers who accepted and refused.
data_scaled_balanced=pd.concat([scaled_data.query('y==1'),scaled_data.query('y==0').sample(n=4640)]).sample(frac=1)

In [55]:
#unbalanced data
Y=scaled_data.iloc[:,0]
X=scaled_data.iloc[:,1:]

In [145]:
#balanced data, X, Y
Y_bal=data_scaled_balanced.iloc[:,0]
X_bal=data_scaled_balanced.iloc[:,1:]

### Problem 6: Train/Test Split

With your data prepared, split it into a train and test set.

In [146]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state = 42)
X_train_bal, X_test_bal, Y_train_bal, Y_test_bal = train_test_split(X_bal, Y_bal, random_state = 42)

In [ ]:
# The above is simple enough.

### Problem 7: A Baseline Model

Before we build our first model, we want to establish a baseline.  What is the baseline performance that our classifier should aim to beat?

#### As we have created a balanced dataset to use in the following experiments with an equal number of successful and unsuccessful calls, we only need to improve an accuracy of 50%

In [249]:
Y_bal.sum()/Y_bal.shape[0]
# the algorithm we need to beat.

0.5

### Problem 8: A Simple Model

Use Logistic Regression to build a basic model on your data.  

In [250]:
# max_iter was increased as the optimization of paramaters was not converging in 100 steps.
lr=LogisticRegression(max_iter=1000)

lr.fit(X_train_bal,Y_train_bal)

LogisticRegression(max_iter=1000)

### Problem 9: Score the Model

What is the accuracy of your model?

In [153]:
preds_lr_bal=lr_bal.predict(X_test_bal)

print(accuracy_score(Y_test_bal,preds_lr_bal))

0.6185344827586207


### Problem 10: Model Comparisons

Now, we aim to compare the performance of the Logistic Regression model to our KNN algorithm, Decision Tree, and SVM models.  Using the default settings for each of the models, fit and score each.  Also, be sure to compare the fit time of each of the models.  Present your findings in a `DataFrame` similar to that below:

| Model | Train Time | Train Accuracy | Test Accuracy |
| ----- | ---------- | -------------  | -----------   |
|     |    |.     |.     |

In [118]:
import time
time.time

1674453011.0579252

In [230]:
def model_comp(model):
    start=time.time()
    model.fit(X_train_bal,Y_train_bal)
    end=time.time()
    train_time=end-start
    train_accuracy=accuracy_score(Y_train_bal,model.predict(X_train_bal))
    test_accuracy=accuracy_score(Y_test_bal,model.predict(X_test_bal))
    return([str(model),train_time,train_accuracy,test_accuracy])
    
    
    

In [231]:
model_comp(DecisionTreeClassifier())


['DecisionTreeClassifier()',
 0.08397984504699707,
 0.8475574712643679,
 0.5543103448275862]

In [232]:
model_comp(LogisticRegression(max_iter=200))

['LogisticRegression(max_iter=200)',
 0.09072399139404297,
 0.5992816091954023,
 0.6112068965517241]

In [233]:
model_comp(KNeighborsClassifier())

['KNeighborsClassifier()',
 0.040426015853881836,
 0.7135057471264368,
 0.5887931034482758]

In [234]:
model_comp(SVC())


['SVC()', 2.2479429244995117, 0.6352011494252874, 0.6017241379310345]

In [235]:
models=[DecisionTreeClassifier(),LogisticRegression(max_iter=200),KNeighborsClassifier(),SVC()]

In [236]:
Basic_Comparisons=pd.DataFrame([model_comp(x) for x in models],columns=['model','train time','train Accuracy','test Accuracy'])

In [246]:
Basic_Comparisons

,model,train time,train Accuracy,test Accuracy
0,DecisionTreeClassifier(),0.043566,0.847557,0.554741
1,LogisticRegression(max_iter=200),0.064937,0.599282,0.611207
2,KNeighborsClassifier(),0.013068,0.713506,0.588793
3,SVC(),1.504857,0.635201,0.601724


### The above DataFrame shows that Knn and Decision Drees are prone to overfitting and do not supply much better accuracy on the test set than SVC or Logistic Regression.

### Problem 11: Improving the Model

Now that we have some basic models on the board, we want to try to improve these.  Below, we list a few things to explore in this pursuit.

- More feature engineering and exploration.  For example, should we keep the gender feature?  Why or why not?
- Hyperparameter tuning and grid search.  All of our models have additional hyperparameters to tune and explore.  For example the number of neighbors in KNN or the maximum depth of a Decision Tree.  
- Adjust your performance metric

## Lets explore Logistic regression with regularization.

### To do this we can iterate over different values of $\alpha$ or $\alpha=\frac{1}{c}$ where $\frac{1}{c}$ is the regularization constant.



In [245]:
logistic_comparison=[]
for c in [1/(10**x) for x in range(10)]:
    
    logistic_comparison.append(model_comp(LogisticRegression(penalty="l2", C=c, max_iter=1000)))


logistic_df=pd.DataFrame(logistic_comparison,columns=['model','train time','train Accuracy','test Accuracy'])   
logistic_df['1/C']=[(10**x) for x in range(10)]
logistic_df

,model,train time,train Accuracy,test Accuracy,1/C
0,LogisticRegression(max_iter=1000),0.097288,0.599282,0.611207,1
1,"LogisticRegression(C=0.1, max_iter=1000)",0.029406,0.602299,0.616810,10
2,"LogisticRegression(C=0.01, max_iter=1000)",0.013885,0.595833,0.615517,100
3,"LogisticRegression(C=0.001, max_iter=1000)",0.008886,0.594828,0.615948,1000
4,"LogisticRegression(C=0.0001, max_iter=1000)",0.005969,0.547557,0.526724,10000
5,"LogisticRegression(C=1e-05, max_iter=1000)",0.005643,0.507615,0.477155,100000
6,"LogisticRegression(C=1e-06, max_iter=1000)",0.005257,0.507615,0.477155,1000000
7,"LogisticRegression(C=1e-07, max_iter=1000)",0.005739,0.507615,0.477155,10000000
8,"LogisticRegression(C=1e-08, max_iter=1000)",0.006744,0.507615,0.477155,100000000
9,"LogisticRegression(C=1e-09, max_iter=1000)",0.007092,0.507615,0.477155,1000000000


### The above cell output shows that a regularization constant of $C=0.001$ provides the best accuracy.

In [254]:
log_reg=LogisticRegression(max_iter=1000,penalty="l2", C=0.001 ).fit(X_train_bal,Y_train_bal)

In [262]:
pd.DataFrame({'feature':list(X_train_bal.columns),'coefficient value':list(log_reg.coef_[0])})

,feature,coefficient value
0,age,0.070456
1,job_admin.,0.028823
2,job_blue-collar,-0.102532
3,job_entrepreneur,-0.009442
4,job_housemaid,-0.009549
5,job_management,-0.023441
6,job_retired,0.078016
7,job_self-employed,-0.011541
8,job_services,-0.029315
9,job_student,0.070209


In [259]:
list(X_train_bal.columns)

['age',
 'job_admin.',
 'job_blue-collar',
 'job_entrepreneur',
 'job_housemaid',
 'job_management',
 'job_retired',
 'job_self-employed',
 'job_services',
 'job_student',
 'job_technician',
 'job_unemployed',
 'job_unknown',
 'marital_divorced',
 'marital_married',
 'marital_single',
 'marital_unknown',
 'education_basic.4y',
 'education_basic.6y',
 'education_basic.9y',
 'education_high.school',
 'education_illiterate',
 'education_professional.course',
 'education_university.degree',
 'education_unknown',
 'default_no',
 'default_unknown',
 'default_yes',
 'housing_no',
 'housing_unknown',
 'housing_yes',
 'loan_no',
 'loan_unknown',
 'loan_yes']

##### Questions